In [2]:
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog
import json
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

class ScrapingRoadmapTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Scraping Roadmap Tool")
        self.root.geometry("600x400")
        
        self.roadmap = []
        self.driver = None
        self.current_url = ""
        
        # Interface principale
        frame = tk.Frame(root)
        frame.pack(pady=20, padx=20, fill="both", expand=True)
        
        # URL input
        url_frame = tk.Frame(frame)
        url_frame.pack(fill="x", pady=10)
        
        tk.Label(url_frame, text="URL du site:").pack(side="left")
        self.url_entry = tk.Entry(url_frame, width=50)
        self.url_entry.pack(side="left", padx=10, fill="x", expand=True)
        
        # Boutons
        button_frame = tk.Frame(frame)
        button_frame.pack(pady=10)
        
        tk.Button(button_frame, text="Ouvrir le site", command=self.open_website).pack(side="left", padx=5)
        tk.Button(button_frame, text="Commencer l'enregistrement", command=self.start_recording).pack(side="left", padx=5)
        tk.Button(button_frame, text="Sauvegarder le roadmap", command=self.save_roadmap).pack(side="left", padx=5)
        
        # Liste des actions enregistrées
        list_frame = tk.Frame(frame)
        list_frame.pack(fill="both", expand=True, pady=10)
        
        tk.Label(list_frame, text="Actions enregistrées:").pack(anchor="w")
        
        self.listbox = tk.Listbox(list_frame, width=70, height=10)
        self.listbox.pack(fill="both", expand=True)
        
        # Bouton pour nommer l'élément sélectionné
        tk.Button(frame, text="Nommer l'élément sélectionné", command=self.name_element).pack(pady=5)
    
    def open_website(self):
        url = self.url_entry.get().strip()
        if not url:
            messagebox.showerror("Erreur", "Veuillez entrer une URL")
            return
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        self.current_url = url
        chrome_options = Options()
        chrome_options.add_experimental_option("detach", True)
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.driver.get(url)
        messagebox.showinfo("Information", "Site ouvert. Cliquez sur 'Commencer l'enregistrement'.")

    def start_recording(self):
        if not self.driver:
            messagebox.showerror("Erreur", "Veuillez d'abord ouvrir un site web")
            return
        # Initialiser tableau d'événements
        init_script = "window._clickRoadmap = [];"
        self.driver.execute_script(init_script)
        js_script = """
        document.addEventListener('click', function(e) {
            function getXPath(el) {
                if (el.id) return '//*[@id="' + el.id + '"]';
                if (el === document.body) return '/html/body';
                var ix=0, siblings=el.parentNode.childNodes;
                for (var i=0; i<siblings.length; i++) {
                    var sib=siblings[i];
                    if (sib===el) {
                        return getXPath(el.parentNode)+'/'+el.tagName.toLowerCase()+'['+(ix+1)+']';
                    }
                    if (sib.nodeType===1 && sib.tagName===el.tagName) ix++;
                }
            }
            function getCss(el) {
                if (el.id) return '#'+el.id;
                if (el.className) return '.'+el.className.trim().split(/\s+/).join('.');
                var path=el.tagName.toLowerCase();
                if (el.parentNode && el.parentNode!==document)
                    path = getCss(el.parentNode)+' > '+path;
                return path;
            }
            var info={
                xpath: getXPath(e.target),
                cssSelector: getCss(e.target),
                tagName: e.target.tagName,
                innerText: (e.target.innerText||'').trim().substring(0,30)
            };
            window._clickRoadmap.push(info);
        }, true);
        """
        self.driver.execute_script(js_script)
        self.root.after(500, self.collect_clicks)
        messagebox.showinfo("Information", "Enregistrement démarré. Cliquez sur le site.")

    def collect_clicks(self):
        if not self.driver: return
        try:
            # Récupérer et vider les événements
            results = self.driver.execute_script(
                "var r=window._clickRoadmap.slice(); window._clickRoadmap=[]; return r;"
            )
            for res in results or []:
                element_info = {
                    'name': f"Element {len(self.roadmap)+1}",
                    'xpath': res.get('xpath',''),
                    'cssSelector': res.get('cssSelector',''),
                    'tagName': res.get('tagName',''),
                    'innerText': res.get('innerText','')
                }
                self.roadmap.append(element_info)
                display = f"{element_info['name']}: {element_info['tagName']} - {element_info['innerText']}"
                self.listbox.insert(tk.END, display)
        except Exception:
            pass
        self.root.after(500, self.collect_clicks)

    def name_element(self):
        sel = self.listbox.curselection()
        if not sel:
            messagebox.showerror("Erreur", "Sélectionnez un élément.")
            return
        idx=sel[0]; cur=self.roadmap[idx]
        new=simpledialog.askstring("Nommer l'élément", "Nouveau nom:", initialvalue=cur['name'])
        if new:
            cur['name']=new
            self.listbox.delete(idx)
            self.listbox.insert(idx, f"{new}: {cur['tagName']} - {cur['innerText']}")

    def save_roadmap(self):
        if not self.roadmap:
            messagebox.showwarning("Avertissement", "Rien à sauvegarder.")
            return
        fp=filedialog.asksaveasfilename(defaultextension='.json', filetypes=[('JSON','*.json')])
        if not fp: return
        try:
            with open(fp,'w',encoding='utf-8') as f:
                json.dump(self.roadmap,f,ensure_ascii=False,indent=4)
            messagebox.showinfo("Succès", f"Enregistré: {fp}")
        except Exception as e:
            messagebox.showerror("Erreur", str(e))

if __name__ == '__main__':
    root=tk.Tk()
    ScrapingRoadmapTool(root)
    root.mainloop()


<>:73: SyntaxWarning: invalid escape sequence '\s'
<>:73: SyntaxWarning: invalid escape sequence '\s'
C:\Users\malek\AppData\Local\Temp\ipykernel_9864\2879159230.py:73: SyntaxWarning: invalid escape sequence '\s'
  js_script = """


In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

class ScrapingRoadmapTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Scraping Roadmap Tool")
        self.root.geometry("650x450")

        self.roadmap = []
        self.driver = None
        self.recording = False

        # UI
        frame = tk.Frame(root)
        frame.pack(padx=20, pady=20, fill="both", expand=True)

        # URL
        url_frame = tk.Frame(frame)
        url_frame.pack(fill="x", pady=5)
        tk.Label(url_frame, text="URL:").pack(side="left")
        self.url_entry = tk.Entry(url_frame)
        self.url_entry.pack(side="left", fill="x", expand=True, padx=5)

        # Buttons open / start / stop / save
        btn_frame = tk.Frame(frame)
        btn_frame.pack(pady=10)
        tk.Button(btn_frame, text="Ouvrir Site", command=self.open_website).pack(side="left", padx=5)
        tk.Button(btn_frame, text="Démarrer Enregistrement", command=self.start_recording).pack(side="left", padx=5)
        tk.Button(btn_frame, text="Arrêter Enregistrement", command=self.stop_recording).pack(side="left", padx=5)
        tk.Button(btn_frame, text="Sauvegarder JSON", command=self.save_roadmap).pack(side="left", padx=5)

        # List of actions
        list_frame = tk.Frame(frame)
        list_frame.pack(fill="both", expand=True)
        tk.Label(list_frame, text="Actions enregistrées:").pack(anchor="w")
        self.listbox = tk.Listbox(list_frame)
        self.listbox.pack(fill="both", expand=True)

        # Rename and Remove
        action_frame = tk.Frame(frame)
        action_frame.pack(pady=5)
        tk.Button(action_frame, text="Renommer", command=self.name_element).pack(side="left", padx=5)
        tk.Button(action_frame, text="Supprimer", command=self.delete_element).pack(side="left", padx=5)

    def open_website(self):
        url = self.url_entry.get().strip()
        if not url:
            messagebox.showerror("Erreur", "Entrez une URL valide.")
            return
        if not url.startswith(('http://','https://')):
            url = 'https://' + url
        options = Options()
        options.add_experimental_option("detach", True)
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        self.driver.get(url)
        messagebox.showinfo("Info", "Site ouvert. Vous pouvez démarrer l'enregistrement.")

    def start_recording(self):
        if not self.driver:
            messagebox.showerror("Erreur", "Ouvrez d'abord le site.")
            return
        if self.recording:
            messagebox.showwarning("Avertissement", "Enregistrement déjà démarré.")
            return
        # inject JS
        self.driver.execute_script("window._cliks=[];")
        js = """
        document.addEventListener('click', function(e){
            var getXPath=function(el){if(el.id)return '//*[@id="'+el.id+'"]';if(el===document.body)return '/html/body';var ix=0,sibs=el.parentNode.childNodes;for(var i=0;i<sibs.length;i++){var s=sibs[i];if(s===el)return getXPath(el.parentNode)+'/'+el.tagName.toLowerCase()+'['+(ix+1)+']';if(s.nodeType===1&&s.tagName===el.tagName)ix++;} };
            var getCss=function(el){if(el.id)return '#'+el.id;if(el.className)return '.'+el.className.trim().split(/\s+/).join('.');var path=el.tagName.toLowerCase();if(el.parentNode&&el.parentNode!==document)path=getCss(el.parentNode)+' > '+path;return path;};
            window._cliks.push({xpath:getXPath(e.target),cssSelector:getCss(e.target),tagName:e.target.tagName,innerText:(e.target.innerText||'').trim().substring(0,30)});
        }, true);
        """
        self.driver.execute_script(js)
        self.recording = True
        self.poll_clicks()
        messagebox.showinfo("Info", "Enregistrement démarré.")

    def poll_clicks(self):
        if not self.recording or not self.driver:
            return
        try:
            items = self.driver.execute_script("var c=window._cliks.slice();window._cliks=[];return c;")
            for it in items or []:
                # avoid duplicates
                if any(r['xpath']==it['xpath'] for r in self.roadmap):
                    continue
                rec={'name':f"Element {len(self.roadmap)+1}",**it}
                self.roadmap.append(rec)
                self.listbox.insert(tk.END,f"{rec['name']}: {rec['cssSelector']} [{rec['tagName']}] - {rec['innerText']}")
        except Exception:
            pass
        self.root.after(400, self.poll_clicks)

    def stop_recording(self):
        if not self.recording:
            messagebox.showwarning("Avertissement", "Aucun enregistrement en cours.")
            return
        self.recording=False
        # remove listener
        self.driver.execute_script("document.removeEventListener('click',arguments.callee,true);")
        messagebox.showinfo("Info", "Enregistrement arrêté.")

    def name_element(self):
        sel=self.listbox.curselection()
        if not sel:
            return
        idx=sel[0];rec=self.roadmap[idx]
        new=simpledialog.askstring("Renommer","Nouveau nom:",initialvalue=rec['name'])
        if new:
            rec['name']=new
            self.listbox.delete(idx)
            self.listbox.insert(idx,f"{new}: {rec['cssSelector']} [{rec['tagName']}] - {rec['innerText']}")

    def delete_element(self):
        sel=self.listbox.curselection()
        if not sel:
            return
        idx=sel[0]
        del self.roadmap[idx]
        self.listbox.delete(idx)
        # renumber
        for i,rec in enumerate(self.roadmap): rec['name']=f"Element {i+1}"
        self.listbox.delete(0,tk.END)
        for rec in self.roadmap:
            self.listbox.insert(tk.END,f"{rec['name']}: {rec['cssSelector']} [{rec['tagName']}] - {rec['innerText']}")

    def save_roadmap(self):
        if not self.roadmap:
            messagebox.showwarning("Avertissement","Rien à sauvegarder.")
            return
        fp=filedialog.asksaveasfilename(defaultextension='.json',filetypes=[('JSON','*.json')])
        if not fp: return
        try:
            with open(fp,'w',encoding='utf-8') as f:
                json.dump(self.roadmap,f,ensure_ascii=False,indent=4)
            messagebox.showinfo("Succès",f"Fichier enregistré: {fp}")
        except Exception as e:
            messagebox.showerror("Erreur",str(e))

if __name__=='__main__':
    root=tk.Tk()
    ScrapingRoadmapTool(root)
    root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


class ScrapingRoadmapTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Scraping Roadmap Tool")
        self.root.geometry("600x400")

        self.roadmap = []
        self.driver = None
        self.current_url = ""

        # Interface principale
        frame = tk.Frame(root)
        frame.pack(pady=20, padx=20, fill="both", expand=True)

        # URL input
        url_frame = tk.Frame(frame)
        url_frame.pack(fill="x", pady=10)

        tk.Label(url_frame, text="URL du site:").pack(side="left")
        self.url_entry = tk.Entry(url_frame, width=50)
        self.url_entry.pack(side="left", padx=10, fill="x", expand=True)

        # Boutons
        button_frame = tk.Frame(frame)
        button_frame.pack(pady=10)

        tk.Button(button_frame, text="Ouvrir le site", command=self.open_website).pack(side="left", padx=5)
        tk.Button(button_frame, text="Commencer l'enregistrement", command=self.start_recording).pack(side="left", padx=5)
        tk.Button(button_frame, text="Sauvegarder le roadmap", command=self.save_roadmap).pack(side="left", padx=5)
        tk.Button(frame, text="Nommer l'élément sélectionné", command=self.name_element).pack(pady=5)

        # Liste des actions enregistrées
        list_frame = tk.Frame(frame)
        list_frame.pack(fill="both", expand=True, pady=10)

        tk.Label(list_frame, text="Actions enregistrées:").pack(anchor="w")

        self.listbox = tk.Listbox(list_frame, width=70, height=10)
        self.listbox.pack(fill="both", expand=True)

        self.preview_frame = tk.Frame(frame)
        self.preview_frame.pack(fill="both", expand=True)

        self.preview_label = tk.Label(self.preview_frame, text="Aperçu de la valeur:")
        self.preview_label.pack(anchor="w")

        self.preview_text = tk.Text(self.preview_frame, height=3, wrap="word")
        self.preview_text.pack(fill="x")

    def show_preview(self, element_info):
        self.preview_text.delete(1.0, tk.END)
        self.preview_text.insert(tk.END, element_info['innerText'] or "<Aucun texte>")

    def open_website(self):
        url = self.url_entry.get()
        if not url:
            messagebox.showerror("Erreur", "Veuillez entrer une URL")
            return

        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url

        self.current_url = url

        chrome_options = Options()
        chrome_options.add_experimental_option("detach", True)

        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.driver.get(url)

        messagebox.showinfo("Information", "Site web ouvert. Utilisez 'Commencer l'enregistrement' pour enregistrer les clics.")

    def start_recording(self):
        if not self.driver:
            messagebox.showerror("Erreur", "Veuillez d'abord ouvrir un site web")
            return

        js_script = """
        document.addEventListener('click', function(e) {
            function getXPath(element) {
                if (element.id !== '') return '//*[@id="' + element.id + '"]';
                if (element === document.body) return '/html/body';
                let ix = 0;
                let siblings = element.parentNode.childNodes;
                for (let i = 0; i < siblings.length; i++) {
                    let sibling = siblings[i];
                    if (sibling === element) {
                        let path = getXPath(element.parentNode);
                        let tag = element.tagName.toLowerCase();
                        return `${path}/${tag}[${ix + 1}]`;
                    }
                    if (sibling.nodeType === 1 && sibling.tagName.toLowerCase() === element.tagName.toLowerCase()) {
                        ix++;
                    }
                }
            }

            function getCssSelector(element) {
    if (element.id) {
        return '#' + CSS.escape(element.id);
    }
    
    let selector = element.tagName.toLowerCase();
    if (element.className) {
        const classes = element.className.trim().split(/\s+/).map(c => CSS.escape(c)).join('.');
        selector += '.' + classes;
    }
    
    // Vérification de l'unicité
    if (document.querySelectorAll(selector).length === 1) {
        return selector;
    }
    
    // Ajout de l'index si nécessaire
    const siblings = Array.from(element.parentNode.children);
    const index = siblings.indexOf(element) + 1;
    return selector + f":nth-of-type({index})";
}

            let xpath = getXPath(e.target);
            let cssSelector = getCssSelector(e.target);
            let innerText = e.target.innerText ? e.target.innerText.substring(0, 30) : '';

            window.callPython = {
                xpath: xpath,
                cssSelector: cssSelector,
                tagName: e.target.tagName,
                innerText: innerText
            };
        });
        """

        self.driver.execute_script(js_script)
        self.check_for_clicks()

        messagebox.showinfo("Information", "Enregistrement démarré. Cliquez sur les éléments du site.")

    def check_for_clicks(self):
    try:
        result = self.driver.execute_script("""
            if (!window.callPython) return null;
            const elem = document.evaluate(
                window.callPython.xpath, 
                document, 
                null, 
                XPathResult.FIRST_ORDERED_NODE_TYPE, 
                null
            ).singleNodeValue;
            return elem ? window.callPython : null;
        """)
        
        if result:
            # Vérification CSS Selector
            try:
                self.driver.find_element(By.CSS_SELECTOR, result["cssSelector"])
            except:
                result["cssSelector"] = None

                element_info = {
                    "name": "Action",
                    "xpath": result["xpath"],
                    "cssSelector": result["cssSelector"],
                    "tagName": result["tagName"],
                    "innerText": result["innerText"]
                }

                self.roadmap.append(element_info)
                display_text = f"{element_info['name']}: {element_info['tagName']} - {element_info['innerText']}"
                self.listbox.insert(tk.END, display_text)
                self.show_preview(element_info)

                self.driver.execute_script("window.callPython = null;")  # reset

        except Exception as e:
            print(f"Erreur dans check_for_clicks: {e}")

        self.root.after(500, self.check_for_clicks)

    def name_element(self):
        selected = self.listbox.curselection()
        if not selected:
            messagebox.showerror("Erreur", "Veuillez sélectionner un élément dans la liste")
            return
        idx = selected[0]
        current = self.roadmap[idx]
        new_name = simpledialog.askstring("Nom de l'élément", "Entrez un nouveau nom pour l'élément:", initialvalue=current['name'])
        if new_name:
            current['name'] = new_name
            self.listbox.delete(idx)
            display_text = f"{current['name']}: {current['tagName']} - {current['innerText']}"
            self.listbox.insert(idx, display_text)

    def save_roadmap(self):
        if not self.roadmap:
            messagebox.showwarning("Avertissement", "Aucune action enregistrée à sauvegarder")
            return

        file_path = filedialog.asksaveasfilename(
            defaultextension='.json',
            filetypes=[('JSON files', '*.json')],
            title='Enregistrer le roadmap'
        )
        if not file_path:
            return

        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(self.roadmap, f, ensure_ascii=False, indent=4)
            messagebox.showinfo("Succès", f"Roadmap sauvegardé dans {file_path}")
        except Exception as e:
            messagebox.showerror("Erreur", f"Impossible de sauvegarder le fichier: {e}")


if __name__ == "__main__":
    root = tk.Tk()
    app = ScrapingRoadmapTool(root)
    root.mainloop()
